<a href="https://colab.research.google.com/github/mishra-yogendra/Hybrid_ChatBot/blob/main/Hybrid_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 7.6 MB/s eta 0:00:00


In [ ]:

NEO4J_URI = " "
NEO4J_USER = " "
NEO4J_PASSWORD = " "

In [ ]:
# load_to_neo4j.py
import json
import logging
from neo4j import GraphDatabase
from tqdm import tqdm
#import config

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

DATA_FILE = "/content/vietnam_travel_dataset.json"

class Neo4jLoader:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_constraints(self):
        with self.driver.session() as session:
            try:
                # Create constraints for each entity type
                constraints = [
                    "CREATE CONSTRAINT IF NOT EXISTS FOR (c:City) REQUIRE c.id IS UNIQUE",
                    "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Attraction) REQUIRE a.id IS UNIQUE",
                    "CREATE CONSTRAINT IF NOT EXISTS FOR (h:Hotel) REQUIRE h.id IS UNIQUE",
                    "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Activity) REQUIRE a.id IS UNIQUE",
                    "CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE e.id IS UNIQUE"
                ]

                for constraint in constraints:
                    session.run(constraint)
                logger.info("Constraints created successfully")

            except Exception as e:
                logger.error(f"Error creating constraints: {e}")
                raise

    def upsert_node_batch(self, nodes_batch):
        """Upsert nodes in batches for better performance"""
        with self.driver.session() as session:
            for node in nodes_batch:
                try:
                    labels = [node.get("type", "Unknown"), "Entity"]
                    label_cypher = ":" + ":".join(labels)

                    # Filter properties to store
                    props = {k: v for k, v in node.items() if k not in ("connections",)}

                    query = f"""
                    MERGE (n{label_cypher} {{id: $id}})
                    SET n += $props
                    RETURN n.id as node_id
                    """

                    result = session.run(query, id=node["id"], props=props)
                    # Consume the result to ensure the query executes
                    result.single()

                except Exception as e:
                    logger.error(f"Error upserting node {node.get('id')}: {e}")

    def create_relationships_batch(self, relationships_batch):
        """Create relationships in batches"""
        with self.driver.session() as session:
            for source_id, rel in relationships_batch:
                try:
                    rel_type = rel.get("relation", "RELATED_TO")
                    target_id = rel.get("target")

                    if not target_id:
                        continue

                    query = f"""
                    MATCH (a:Entity {{id: $source_id}}), (b:Entity {{id: $target_id}})
                    MERGE (a)-[r:{rel_type}]->(b)
                    RETURN type(r) as relationship_type
                    """

                    result = session.run(query, source_id=source_id, target_id=target_id)
                    result.single()

                except Exception as e:
                    logger.error(f"Error creating relationship {source_id} -> {target_id}: {e}")

    def load_data(self, batch_size=100):
        """Main method to load data with batch processing"""
        try:
            # Load JSON data
            with open(DATA_FILE, "r", encoding="utf-8") as f:
                nodes = json.load(f)

            logger.info(f"Loaded {len(nodes)} nodes from {DATA_FILE}")

            # Create constraints first
            self.create_constraints()

            # Process nodes in batches
            logger.info("Upserting nodes...")
            for i in tqdm(range(0, len(nodes), batch_size), desc="Processing node batches"):
                batch = nodes[i:i + batch_size]
                self.upsert_node_batch(batch)

            # Process relationships in batches
            logger.info("Creating relationships...")
            relationships = []
            for node in nodes:
                conns = node.get("connections", [])
                for rel in conns:
                    relationships.append((node["id"], rel))

            for i in tqdm(range(0, len(relationships), batch_size), desc="Processing relationship batches"):
                batch = relationships[i:i + batch_size]
                self.create_relationships_batch(batch)

            logger.info("Data loading completed successfully!")

        except Exception as e:
            logger.error(f"Error loading data: {e}")
            raise

    def verify_data(self):
        """Verify the loaded data by counting nodes and relationships"""
        with self.driver.session() as session:
            # Count nodes by type
            node_counts = session.run("""
            MATCH (n)
            RETURN labels(n)[0] as type, count(n) as count
            ORDER BY type
            """)

            logger.info("Node counts by type:")
            for record in node_counts:
                logger.info(f"  {record['type']}: {record['count']}")

            # Count relationships by type
            rel_counts = session.run("""
            MATCH ()-[r]->()
            RETURN type(r) as relationship_type, count(r) as count
            ORDER BY relationship_type
            """)

            logger.info("Relationship counts by type:")
            for record in rel_counts:
                logger.info(f"  {record['relationship_type']}: {record['count']}")

def main():
    loader = None
    try:
        # Initialize loader
        loader = Neo4jLoader(NEO4J_URI,NEO4J_USER,NEO4J_PASSWORD)

        # Load data
        loader.load_data(batch_size=50)  # Adjust batch size as needed

        # Verify data
        loader.verify_data()

    except Exception as e:
        logger.error(f"Failed to load data: {e}")

    finally:
        if loader:
            loader.close()

if __name__ == "__main__":
    main()

Processing relationship batches: 100%|██████████| 8/8 [01:04<00:00,  8.11s/it]


In [ ]:
PINECONE_API_KEY = " " # your Pinecone API key
PINECONE_CLOUD = "aws"          # serverless cloud provider
PINECONE_ENV = "us-east-1"      # serverless region (e.g., us-east-1, eu-west-1)
PINECONE_INDEX_NAME = "vietnam-travel"
PINECONE_VECTOR_DIM = 384

In [ ]:
%pip install pinecone

In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
# pinecone_upload_hf_groq.py
import json
import time
from tqdm import tqdm
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from groq import Groq
import os

# -----------------------------
# Config - Update these with your actual credentials
# -----------------------------
DATA_FILE = "vietnam_travel_dataset.json"
BATCH_SIZE = 32


# Hugging Face Config
HF_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # 384 dimensions
# Alternative models you can use:
# "sentence-transformers/all-mpnet-base-v2" (768 dimensions)
# "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" (multilingual, 384 dimensions)

# Groq Config
GROQ_API_KEY = " "
GROQ_MODEL = "llama2-70b-4096"  # or "mixtral-8x7b-32768"

# -----------------------------
# Initialize clients
# -----------------------------
print("Loading Hugging Face model...")
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)
model = AutoModel.from_pretrained(HF_MODEL_NAME)

# Initialize Groq client for semantic text enhancement (optional)
groq_client = Groq(api_key=GROQ_API_KEY)

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Get embedding dimension from model
with torch.no_grad():
    # Get model embedding dimension
    test_embedding = model(**tokenizer("test", return_tensors="pt", truncation=True))
    VECTOR_DIM = test_embedding.last_hidden_state.size(-1)

print(f"Model embedding dimension: {VECTOR_DIM}")

# -----------------------------
# Create managed index if it doesn't exist
# -----------------------------
existing_indexes = pc.list_indexes().names()
if PINECONE_INDEX_NAME not in existing_indexes:
    print(f"Creating managed index: {PINECONE_INDEX_NAME} with dimension {VECTOR_DIM}")
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=VECTOR_DIM,
        metric="cosine",
        spec=ServerlessSpec(
            cloud=PINECONE_CLOUD,
            region=PINECONE_ENV
        )
    )
    # Wait for index to be ready
    time.sleep(10)
else:
    print(f"Index {PINECONE_INDEX_NAME} already exists.")

# Connect to the index
index = pc.Index(PINECONE_INDEX_NAME)

# -----------------------------
# Helper functions
# -----------------------------
def get_hf_embeddings(texts):
    """Generate embeddings using Hugging Face model."""
    # Tokenize the texts
    encoded_input = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Mean pooling to get sentence embeddings
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return embeddings.numpy()

def mean_pooling(model_output, attention_mask):
    """Apply mean pooling to get sentence embeddings."""
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def enhance_semantic_text_groq(original_text, node_data):
    """Use Groq LLM to enhance semantic text (optional)."""
    try:
        prompt = f"""
        Given the following travel data about Vietnam, create a comprehensive semantic description for vector search:

        Original Text: {original_text}
        Name: {node_data.get('name', '')}
        Type: {node_data.get('type', '')}
        City: {node_data.get('city', node_data.get('region', ''))}
        Tags: {', '.join(node_data.get('tags', []))}
        Description: {node_data.get('description', '')[:500]}

        Create a rich, search-optimized description that captures the essence of this travel destination/activity.
        Focus on key features, experiences, and what makes it unique for travelers.
        """

        response = groq_client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=GROQ_MODEL,
            max_tokens=200,
            temperature=0.3
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Groq enhancement failed: {e}")
        return original_text

def chunked(iterable, n):
    """Split iterable into chunks of size n."""
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

# -----------------------------
# Main upload function
# -----------------------------
def main(enhance_with_groq=False):
    """Main function to process and upload data to Pinecone."""

    # Load data
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        nodes = json.load(f)

    print(f"Loaded {len(nodes)} nodes from {DATA_FILE}")

    # Prepare items for upload
    items = []
    for node in tqdm(nodes, desc="Preparing items"):
        # Get or create semantic text
        semantic_text = node.get("semantic_text") or node.get("description", "")[:1000]

        if not semantic_text.strip():
            continue

        # Optional: Enhance semantic text with Groq
        if enhance_with_groq:
            try:
                semantic_text = enhance_semantic_text_groq(semantic_text, node)
                time.sleep(0.5)  # Rate limiting
            except Exception as e:
                print(f"Failed to enhance text for {node['id']}: {e}")

        # Prepare metadata
        meta = {
            "id": node.get("id"),
            "type": node.get("type"),
            "name": node.get("name"),
            "city": node.get("city", node.get("region", "")),
            "tags": node.get("tags", []),
            "description": node.get("description", "")[:500],
            "best_time_to_visit": node.get("best_time_to_visit", ""),
            "region": node.get("region", "")
        }

        # Clean metadata (remove empty fields)
        meta = {k: v for k, v in meta.items() if v}

        items.append((node["id"], semantic_text, meta))

    print(f"Prepared {len(items)} items for upload to Pinecone...")

    # Upload in batches
    successful_uploads = 0
    for batch in tqdm(list(chunked(items, BATCH_SIZE)), desc="Uploading batches"):
        try:
            ids = [item[0] for item in batch]
            texts = [item[1] for item in batch]
            metas = [item[2] for item in batch]

            # Get embeddings from Hugging Face model
            embeddings = get_hf_embeddings(texts)

            # Prepare vectors for Pinecone
            vectors = [
                {
                    "id": _id,
                    "values": emb.tolist() if hasattr(emb, 'tolist') else emb,
                    "metadata": meta
                }
                for _id, emb, meta in zip(ids, embeddings, metas)
            ]

            # Upload to Pinecone
            index.upsert(vectors=vectors)
            successful_uploads += len(batch)

            # Small delay to avoid rate limits
            time.sleep(0.1)

        except Exception as e:
            print(f"Error uploading batch: {e}")
            continue

    print(f"Upload completed! {successful_uploads}/{len(items)} items uploaded successfully.")

    # Print index stats
    try:
        stats = index.describe_index_stats()
        print(f"\nIndex Stats: {stats}")
    except Exception as e:
        print(f"Could not get index stats: {e}")

# -----------------------------
# Alternative version with different Hugging Face models
# -----------------------------
def initialize_model(model_name):
    """Initialize different Hugging Face models."""
    print(f"Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    return tokenizer, model

# Available models with their dimensions
AVAILABLE_MODELS = {
    "miniLM": ("sentence-transformers/all-MiniLM-L6-v2", 384),
    "mpnet": ("sentence-transformers/all-mpnet-base-v2", 768),
    "multilingual": ("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", 384),
    "distilroberta": ("sentence-transformers/all-distilroberta-v1", 768)
}

if __name__ == "__main__":
    # Choose your model (options: miniLM, mpnet, multilingual, distilroberta)
    selected_model = "miniLM"  # Change this to try different models

    if selected_model in AVAILABLE_MODELS:
        HF_MODEL_NAME, VECTOR_DIM = AVAILABLE_MODELS[selected_model]
        tokenizer, model = initialize_model(HF_MODEL_NAME)
    else:
        raise ValueError(f"Model {selected_model} not found. Available: {list(AVAILABLE_MODELS.keys())}")

    # Run upload (set enhance_with_groq=True to use Groq for text enhancement)
    main(enhance_with_groq=False)

Loading Hugging Face model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Model embedding dimension: 384
Index vietnam-travel already exists.
Loading model: sentence-transformers/all-MiniLM-L6-v2
Loaded 360 nodes from vietnam_travel_dataset.json


Preparing items: 100%|██████████| 360/360 [00:00<00:00, 321402.61it/s]


Prepared 360 items for upload to Pinecone...


Uploading batches: 100%|██████████| 12/12 [00:08<00:00,  1.36it/s]

Upload completed! 360/360 items uploaded successfully.

Index Stats: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 360}},
 'total_vector_count': 360,
 'vector_type': 'dense'}


In [ ]:
%pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.9 MB/s eta 0:00:00


In [ ]:
# visualize_graph.py
from neo4j import GraphDatabase
from pyvis.network import Network
import networkx as nx
#import config

NEO_BATCH = 500  # number of relationships to fetch / visualize

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def fetch_subgraph(tx, limit=500):
    # fetch nodes and relationships up to a limit
    q = (
        "MATCH (a:Entity)-[r]->(b:Entity) "
        "RETURN a.id AS a_id, labels(a) AS a_labels, a.name AS a_name, "
        "b.id AS b_id, labels(b) AS b_labels, b.name AS b_name, type(r) AS rel "
        "LIMIT $limit"
    )
    return list(tx.run(q, limit=limit))

def build_pyvis(rows, output_html="neo4j_viz.html"):
    net = Network(height="900px", width="100%", notebook=False, directed=True)
    for rec in rows:
        a_id = rec["a_id"]; a_name = rec["a_name"] or a_id
        b_id = rec["b_id"]; b_name = rec["b_name"] or b_id
        a_labels = rec["a_labels"]; b_labels = rec["b_labels"]
        rel = rec["rel"]

        net.add_node(a_id, label=f"{a_name}\n({','.join(a_labels)})", title=f"{a_name}")
        net.add_node(b_id, label=f"{b_name}\n({','.join(b_labels)})", title=f"{b_name}")
        net.add_edge(a_id, b_id, title=rel)

    net.show(output_html, notebook=False)
    print(f"Saved visualization to {output_html}")

def main():
    with driver.session() as session:
        rows = session.execute_read(fetch_subgraph, limit=NEO_BATCH)
    build_pyvis(rows)

if __name__ == "__main__":
    main()


neo4j_viz.html
Saved visualization to neo4j_viz.html


In [ ]:
# hybrid_chat.py
import json
from typing import List
from groq import Groq
from pinecone import Pinecone, ServerlessSpec
from neo4j import GraphDatabase
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
#import config

# -----------------------------
# Config
# -----------------------------
# EMBED_MODEL = "text-embedding-3-small" # Not available with Groq
# Hugging Face Config (using the same as in the upload script)
HF_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2" # 384 dimensions
VECTOR_DIM = 384 # Must match the dimension of the Pinecone index

# Groq Config
GROQ_API_KEY = " "
CHAT_MODEL = "openai/gpt-oss-20b"
#CHAT_MODEL = "gpt-4o-mini"
TOP_K = 5

INDEX_NAME = PINECONE_INDEX_NAME

# -----------------------------
# Initialize clients
# -----------------------------
client = Groq(api_key=GROQ_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)

# Initialize Hugging Face model for embeddings
print("Loading Hugging Face model for embeddings...")
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)
model = AutoModel.from_pretrained(HF_MODEL_NAME)

# Connect to Pinecone index
if INDEX_NAME not in pc.list_indexes().names():
    print(f"Creating managed index: {INDEX_NAME}")
    pc.create_index(
        name=INDEX_NAME,
        dimension=VECTOR_DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud=PINECONE_CLOUD, region=PINECONE_ENV)
    )

index = pc.Index(INDEX_NAME)

# Connect to Neo4j
driver = GraphDatabase.driver(
         NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD)
)

# -----------------------------
# Helper functions
# -----------------------------
def mean_pooling(model_output, attention_mask):
    """Apply mean pooling to get sentence embeddings."""
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_hf_embeddings(texts):
    """Generate embeddings using Hugging Face model."""
    # Tokenize the texts
    encoded_input = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Mean pooling to get sentence embeddings
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return embeddings.numpy().tolist() # Return as list for Pinecone

def embed_text(text: str) -> List[float]:
    """Get embedding for a text string using Hugging Face model."""
    return get_hf_embeddings([text])[0]


def pinecone_query(query_text: str, top_k=TOP_K):
    """Query Pinecone index using embedding."""
    vec = embed_text(query_text)
    res = index.query(
        vector=vec,
        top_k=top_k,
        include_metadata=True,
        include_values=False
    )
    print("DEBUG: Pinecone top 5 results:")
    print(len(res["matches"]))
    return res["matches"]

def fetch_graph_context(node_ids: List[str], neighborhood_depth=1):
    """Fetch neighboring nodes from Neo4j."""
    facts = []
    with driver.session() as session:
        for nid in node_ids:
            q = (
                "MATCH (n:Entity {id:$nid})-[r]-(m:Entity) "
                "RETURN type(r) AS rel, labels(m) AS labels, m.id AS id, "
                "m.name AS name, m.type AS type, m.description AS description "
                "LIMIT 10"
            )
            recs = session.run(q, nid=nid)
            for r in recs:
                facts.append({
                    "source": nid,
                    "rel": r["rel"],
                    "target_id": r["id"],
                    "target_name": r["name"],
                    "target_desc": (r["description"] or "")[:400],
                    "labels": r["labels"]
                })
    print("DEBUG: Graph facts:")
    print(len(facts))
    return facts

def build_prompt(user_query, pinecone_matches, graph_facts):
    """Build a chat prompt combining vector DB matches and graph facts."""
    system = (
        "You are a helpful travel assistant. Use the provided semantic search results "
        "and graph facts to answer the user's query briefly and concisely. "
        "Cite node ids when referencing specific places or attractions."
    )

    vec_context = []
    for m in pinecone_matches:
        meta = m["metadata"]
        score = m.get("score", None)
        snippet = f"- id: {m['id']}, name: {meta.get('name','')}, type: {meta.get('type','')}, score: {score}"
        if meta.get("city"):
            snippet += f", city: {meta.get('city')}"
        vec_context.append(snippet)

    graph_context = [
        f"- ({f['source']}) -[{f['rel']}]-> ({f['target_id']}) {f['target_name']}: {f['target_desc']}"
        for f in graph_facts
    ]

    prompt = [
        {"role": "system", "content": system},
        {"role": "user", "content":
         f"User query: {user_query}\n\n"
         "Top semantic matches (from vector DB):\n" + "\n".join(vec_context[:10]) + "\n\n"
         "Graph facts (neighboring relations):\n" + "\n".join(graph_context[:20]) + "\n\n"
         "Based on the above, answer the user's question. If helpful, suggest 2–3 concrete itinerary steps or tips and mention node ids for references."}
    ]
    return prompt

def call_chat(prompt_messages):
    """Call OpenAI ChatCompletion."""
    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=prompt_messages,
        max_tokens=600,
        temperature=0.2
    )
    return resp.choices[0].message.content

# -----------------------------
# Interactive chat
# -----------------------------
def interactive_chat():
    print("Hybrid travel assistant. Type 'exit' to quit.")
    while True:
        query = input("\nEnter your travel question: ").strip()
        if not query or query.lower() in ("exit","quit"):
            break

        matches = pinecone_query(query, top_k=TOP_K)
        match_ids = [m["id"] for m in matches]
        graph_facts = fetch_graph_context(match_ids)
        prompt = build_prompt(query, matches, graph_facts)
        answer = call_chat(prompt)
        print("\n=== Assistant Answer ===\n")
        print(answer)
        print("\n=== End ===\n")

if __name__ == "__main__":
    interactive_chat()

Loading Hugging Face model for embeddings...
Hybrid travel assistant. Type 'exit' to quit.

Enter your travel question: create a romantic 4 day itinerary for Vietnam
DEBUG: Pinecone top 5 results:
5
DEBUG: Graph facts:
23

=== Assistant Answer ===

**4‑Day Romantic Itinerary in Vietnam**

| Day | Destination | Highlights (node IDs) | Romantic Touch |
|-----|-------------|------------------------|----------------|
| 1 | **Hanoi** | • Explore the Old Quarter & Hoàn Kiếm Lake (attraction_1, attraction_2) <br>• Visit the Temple of Literature (attraction_4) | Sunset walk around Hoàn Kiếm – perfect for a quiet moment together. |
| 2 | **Hanoi** | • Take a cyclo tour of the French Quarter (activity_27) <br>• Enjoy a water‑puppet show (activity_28) <

=== End ===


Enter your travel question: Create a 3-day itinerary for Hue including attractions, a hotel, and two activities.
DEBUG: Pinecone top 5 results:
5
DEBUG: Graph facts:
14

=== Assistant Answer ===

**3‑Day Hue Itinerary**

| Day | Mor